# Data Preprocessing

In [ ]:
import os

In [ ]:
root_dir = "/kaggle/input/cityscapes/Cityspaces"
input_dir = os.path.join(root_dir, 'images')
output_dir = os.path.join(root_dir, 'gtFine')

input_train = []
output_train = []
input_val = []
output_val = []

In [ ]:
for split in ['train', 'val']:
    input_split_dir = os.path.join(input_dir, split)
    output_split_dir = os.path.join(output_dir, split)
    
    for subdir, _, files in os.walk(input_split_dir):
        for file in files:
            if file.endswith('_leftImg8bit.png'):
                image_path = os.path.join(subdir, file)
                
                # Derive the corresponding output path
                relative_subdir = os.path.relpath(subdir, input_split_dir)
                base_filename = file.replace('_leftImg8bit.png', '_gtFine_color.png')
                output_path = os.path.join(output_split_dir, relative_subdir, base_filename)
                
                # Check if the output path exists
                if os.path.exists(output_path):
                    if split == 'train':
                        input_train.append(image_path)
                        output_train.append(output_path)
                    elif split == 'val':
                        input_val.append(image_path)
                        output_val.append(output_path)
                else:
                    print(f"Output not found for {image_path}, skipping.")

In [ ]:
len(input_train), len(output_train), len(input_val), len(output_val) 

In [ ]:
import cv2
import matplotlib.pyplot as plt


for i in range(3):
    input_img = cv2.imread(input_train[i])
    output_img = cv2.imread(output_train[i])
    
    input_img_rgb = cv2.cvtColor(input_img, cv2.COLOR_BGR2RGB)
    output_img_rgb = cv2.cvtColor(output_img, cv2.COLOR_BGR2RGB)
    
    plt.figure(figsize=(10, 5))
    
    plt.subplot(1, 2, 1)
    plt.imshow(input_img_rgb)
    plt.title('Input Image')
    plt.axis('off')
    
    plt.subplot(1, 2, 2)
    plt.imshow(output_img_rgb)
    plt.title('Output Image')
    plt.axis('off')
    
    plt.show()

In [ ]:
import numpy as np

test_image = cv2.imread(output_train[3])
test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)
unique_colors_original = np.unique(test_image.reshape(-1, test_image.shape[2]), axis=0)
print(f"Unique colors in the original image: {unique_colors_original}")
plt.imshow(test_image)
plt.axis('off')

In [ ]:
test_image_resized = cv2.resize(test_image, (256, 128), interpolation=cv2.INTER_NEAREST)
unique_colors_resized = np.unique(test_image_resized.reshape(-1, test_image_resized.shape[2]), axis=0)
print(f"Unique colors in the resized image: {unique_colors_resized}")
plt.imshow(test_image_resized)
plt.axis('off')

In [ ]:
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

# Function to read and process each image
def process_image(output_path):
    output_img = cv2.imread(output_path)
    output_img = cv2.cvtColor(output_img, cv2.COLOR_BGR2RGB)
    output_img = cv2.resize(output_img, (256, 128), interpolation=cv2.INTER_NEAREST)
    pixels = output_img.reshape(-1, output_img.shape[2])
    return np.unique(pixels, axis=0)

# Initialize an empty set to store unique colors
unique_colors = set()

# Combine output_train and output_val into one list for processing
all_output_paths = output_train + output_val

# Use ThreadPoolExecutor for concurrent image processing
with ThreadPoolExecutor() as executor:
    # Create a generator for the futures
    futures = [executor.submit(process_image, path) for path in all_output_paths]

    # Iterate over futures and collect unique colors
    for future in tqdm(futures, desc="Processing Images"):
        unique_colors_in_image = future.result()
        unique_colors.update(map(tuple, unique_colors_in_image))

# After processing all images, print the unique colors
print(unique_colors)

In [ ]:
len(unique_colors)

In [ ]:
print(sorted(unique_colors))

In [ ]:
import pandas as pd

sorted_colors = sorted(unique_colors)

# Based on https://github.com/mcordts/cityscapesScripts/blob/master/cityscapesscripts/helpers/labels.py
sorted_labels = ['unlabeled', 'truck', 'caravan', 'trailer', 'car', 'motorcycle',
                'bus', 'train', 'building', 'sky', 'ground', 'wall', 'vegetation',
                'dynamic', 'bicycle', 'road', 'bridge', 'tunnel', 'terrain', 'pole',
                'guard rail', 'fence', 'person', 'traffic sign', 'rail track', 'sidewalk',
                'traffic light', 'parking', 'rider']

labels_data = {
    'id': range(len(sorted_colors)),
    'label': sorted_labels,
    'rgb': sorted_colors
}

labels_df = pd.DataFrame(data)
labels_df.to_csv('labels_data.csv', index=False)
labels_df

In [ ]:
rgb_to_id = {tuple(color): idx for idx, color in zip(labels_df['id'], labels_df['rgb'])}

def load_data(input_paths, output_paths):
    X = []
    Y = []
    
    for img_path in tqdm(input_paths, desc="Loading Images"):
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (256, 128), interpolation=cv2.INTER_LINEAR)
        X.append(img)
    
    for mask_path in tqdm(output_paths, desc="Loading Masks"):
        mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
        mask = cv2.resize(mask, (256, 128), interpolation=cv2.INTER_NEAREST)
        pixel_ids = np.zeros((128, 256), dtype=np.uint8)

        for color, idx in rgb_to_id.items():
            binary_mask = np.all(mask[:, :, :3] == np.array(color), axis=-1)
            pixel_ids[binary_mask] = idx
        
        Y.append(pixel_ids)

    X = np.array(X)
    Y = np.array(Y)

    return X, Y

In [ ]:
X_train, Y_train = load_data(input_train, output_train)

In [ ]:
X_train.shape, Y_train.shape

In [ ]:
# Sample index
sample_index = 10

# Create color mask for the label
color_mask = np.zeros((Y_train[sample_index].shape[0], Y_train[sample_index].shape[1], 3), dtype=np.uint8)
for idx, row in labels_df.iterrows():
    color = row['rgb']
    color_mask[Y_train[sample_index] == row['id']] = color

# Plot the image and its label
plt.figure(figsize=(10, 5))

# Plotting the image
plt.subplot(1, 2, 1)
plt.imshow(X_train[sample_index])
plt.title("Sample Image")
plt.axis('off')

# Plotting the label
plt.subplot(1, 2, 2)
plt.imshow(color_mask)
plt.title("Label Mask")
plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
X_val, Y_val = load_data(input_val, output_val)

In [ ]:
X_val.shape, Y_val.shape

In [ ]:
np.save('X_train.npy', X_train)
np.save('Y_train.npy', Y_train)
np.save('X_val.npy', X_val)
np.save('Y_val.npy', Y_val)

---